# AutoGluon - Predicción de ventas (tn) por producto para febrero 2020

In [1]:
import pandas as pd
import os

In [2]:
# import multiprocessing as mp

# # fuerza fork como método de arranque de procesos
# try:
#     mp.set_start_method("fork", force=True)
# except RuntimeError:
#     # si ya se había establecido otro método, lo ignoramos
#     pass

In [5]:
BASE_DATOS     = '/Users/indianaabeledo/Documents/Maestria/laboratorio_III/datasets/'
INTERMEDIOS   = '/Users/indianaabeledo/Documents/Maestria/laboratorio_III/entrega_final/intermedios/'
BASE_OUTPUTS   = '/Users/indianaabeledo/Documents/Maestria/laboratorio_III/entrega_final/output/'

# Crear carpetas si no existen
os.makedirs(INTERMEDIOS, exist_ok=True)
os.makedirs(BASE_OUTPUTS, exist_ok=True)

PRED_PATH = os.path.join(BASE_OUTPUTS,'pred_modelo_autogluon.csv')

In [6]:
# Carga de archivos desde Drive
print("🔄 Cargando archivos...")
productos_pred = pd.read_csv(os.path.join(BASE_DATOS, "productos_pred.txt"), sep="\t")
df_sellin      = pd.read_csv(os.path.join(BASE_DATOS, "sell-in.txt"),        sep="\t")
df_productos   = pd.read_csv(os.path.join(BASE_DATOS, "tb_productos.txt"),   sep="\t")

🔄 Cargando archivos...


In [7]:
# 📄 Leer lista de productos a predecir
# 1. Asegurar de que sean enteros
productos_pred['product_id'] = productos_pred['product_id'].astype(int)

# 2. Extrae la lista
product_ids = productos_pred['product_id'].tolist()

# 🧹 3. Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

# Filtrar hasta dic 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

In [8]:
# ⚙️ 5. Definir y entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start),
)

predictor.fit(ts_data, num_val_windows=2, time_limit=60*60)

# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01'] # Colocar mes a predecir '2020-02-01'

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# === 8. Guardar en CSV usando PRED_PATH ===
resultado.to_csv(PRED_PATH, index=False)
print(f"✅ Guardado en: {PRED_PATH}")

Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to '/Users/indianaabeledo/Documents/Maestria/laboratorio_III/entrega_final/AutogluonModels/ag-20250715_231710'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.6
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:29 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T6030
CPU Count:          11
GPU Count:          0
Memory Avail:       2.87 GB / 18.00 GB (15.9%)
Disk Space Avail:   322.18 GB / 460.43 GB (70.0%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'ti

Index(['item_id', 'timestamp', 'mean'], dtype='object')
✅ Guardado en: /Users/indianaabeledo/Documents/Maestria/laboratorio_III/entrega_final/output/predicciones_modelo_Autogluonv2.csv


# Errores 201912

In [ ]:
# df_monthly tiene [timestamp, item_id, tn, …]
df_cut = df_monthly_product.loc[
    df_monthly_product['timestamp'] <= '2019-10-01'
].copy()

In [ ]:
# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_cut,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

Name: autogluon.timeseries
Version: 1.3.1
Summary: Fast and Accurate ML in 3 Lines of Code
Home-page: https://github.com/autogluon/autogluon
Author: AutoGluon Community
Author-email: 
License: Apache-2.0
Location: /Users/indianaabeledo/Library/Python/3.9/lib/python/site-packages
Requires: scipy, mlforecast, pytorch-lightning, orjson, tensorboard, fugue, networkx, autogluon.features, torch, coreforecast, accelerate, joblib, transformers, lightning, autogluon.tabular, gluonts, tqdm, pandas, utilsforecast, autogluon.core, statsforecast, autogluon.common, numpy
Required-by: autogluon
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ⚙️ 5. Definir y entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start),
)

predictor.fit(ts_data, num_val_windows=2, time_limit=60*60)

# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)


# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2019-12-01']#Colocar mes a predecir '2020-02-01'

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

In [ ]:
# 1) Filtrar valores reales de diciembre-2019
actual = (
    df_monthly_product
      .query("timestamp == '2019-12-01'")
      .loc[:, ['product_id', 'tn']]
      .rename(columns={'tn': 'tn_real'})
)

# 2) Unir predicción y real
#    'resultado' debe tener columnas ['product_id','tn'] donde 'tn' es la predicha.
df_comp = actual.merge(resultado, on='product_id', how='left')

# 3) (Opcional) Agregar / sumar por product_id si hubiera varias filas
df_agg = (
    df_comp
      .groupby('product_id')[['tn_real', 'tn']]
      .sum()
      .reset_index()
)

# 4) Calcular errores
df_agg['abs_error'] = (df_agg['tn_real'] - df_agg['tn']).abs()
df_agg['pct_error'] = df_agg['abs_error'] / df_agg['tn_real']

In [ ]:
 #Definir ruta de salida (reusa BASE_OUTPUTS si ya la tienes)
ERROR_CSV_PATH = os.path.join(BASE_OUTPUTS, 'error_201912_autogluon.csv')

# Exportar a CSV
df_agg.to_csv(ERROR_CSV_PATH, index=False)

print(f"✅ Resultados exportados a: {ERROR_CSV_PATH}")

In [ ]:
resultado.head()

,product_id,tn
0,20001,1318.842610
1,20002,1088.935465
2,20003,718.753417
3,20004,546.314163
4,20005,529.565386
